In [1]:
import pandas as pd
import os
import sys
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Access the API key
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
EMBED_DIM = 3072

In [2]:
df = pd.read_csv('support.csv')

In [3]:
df.head()

,Unnamed: 0,title,link,content
0,0,Tài khoản,https://hotro.hasaki.vn/tai-khoan.html,Đăng ký thành viên Hasaki như thế nào? Quý khá...
1,1,Đặt hàng trực tuyến,https://hotro.hasaki.vn/dat-hang-tai-hasaki.html,Quý khách có thể đặt hàng qua điện thoại bằng ...
2,2,Quy cách đóng gói sản phẩm,https://hotro.hasaki.vn/quy-cach-dong-goi.html,Quy cách đóng gói Hướng dẫn đặt hàng Đặt dịch ...
3,3,Vận chuyển 2H,https://hotro.hasaki.vn/van-chuyen-2h.html,Vận chuyển 2H tại Hasaki là dịch vụ giao hàng ...
4,4,Phí vận chuyển,https://hotro.hasaki.vn/phi-van-chuyen.html,Phí vận chuyển khi đặt hàng tại Hasaki.vn Hasa...


In [5]:
from pinecone_connection import PineConeDB

hasaki_index = PineConeDB(PINECONE_API_KEY).pc.Index('hasaki-index-v3')

In [6]:
from pc_utils import create_vector_emb

In [7]:
df['vector'] = df['title'].apply(lambda text: create_vector_emb(text))

In [9]:
df.head()

,Unnamed: 0,title,link,content,vector
0,0,Tài khoản,https://hotro.hasaki.vn/tai-khoan.html,Đăng ký thành viên Hasaki như thế nào? Quý khá...,"[0.011986899, -0.0447314, 0.0018757733, 0.0144..."
1,1,Đặt hàng trực tuyến,https://hotro.hasaki.vn/dat-hang-tai-hasaki.html,Quý khách có thể đặt hàng qua điện thoại bằng ...,"[-0.028532239, -0.008807353, 0.011129365, -0.0..."
2,2,Quy cách đóng gói sản phẩm,https://hotro.hasaki.vn/quy-cach-dong-goi.html,Quy cách đóng gói Hướng dẫn đặt hàng Đặt dịch ...,"[0.003810922, -0.04745973, 0.0039359285, 0.023..."
3,3,Vận chuyển 2H,https://hotro.hasaki.vn/van-chuyen-2h.html,Vận chuyển 2H tại Hasaki là dịch vụ giao hàng ...,"[-0.030840136, -0.025272, -0.00039213282, -0.0..."
4,4,Phí vận chuyển,https://hotro.hasaki.vn/phi-van-chuyen.html,Phí vận chuyển khi đặt hàng tại Hasaki.vn Hasa...,"[-0.028732294, -0.02889809, 0.0013387956, -0.0..."


In [11]:
upsert_data = []
for idx, row in df.iterrows():
    vector = row['vector']
    metadata = {
        "title": row['title'],
        "link": row['link'],
        "content": row['content']
    }
    upsert_data.append((str(idx), vector, metadata))

In [12]:
hasaki_index.upsert(vectors=upsert_data, namespace='support-namespace')

{'upserted_count': 26}